<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/4_analysis_hate_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Analisis del Dataset completo marzo 2020 - abril 2021 de X

**Natalia Dedandi**

Se aplica el clasificador seleccionado a todo el conjunto de datos recoelctado para analizar las características del odio en Argentina durante la pandemia

In [15]:
!pip install datasets seaborn


Recupero el conjutno completo de datos: https://huggingface.co/datasets/piuba-bigdata/articles_and_comments



In [16]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("piubamas/articles_and_comments")


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
        num_rows: 537201
    })
})

In [18]:
ds1 = dataset["train"]

Ejemplo de una fila

In [20]:
ds1[122]

{'tweet_id': '1376979520729849858',
 'text': 'Una fuerte apuesta a la rueda de la agroindustrialización argentina https://t.co/1o6ThCdtxw https://t.co/TLfGhLBjHu',
 'title': 'Una fuerte apuesta a la rueda de la agroindustrialización argentina',
 'url': 'https://www.lanacion.com.ar/economia/campo/una-fuerte-apuesta-a-la-rueda-de-la-agroindustrializacion-argentina-nid30032021/?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1617130198',
 'user': 'LANACION',
 'body': 'Desde el primer día, YPF Agro se propuso proveer al campo argentino de los insumos y las energías necesarias para producir más y mejor, en un contexto donde la demanda de alimentos para la población mundial sigue creciendo.\n\nEl negocio de agro de YPF cerró 2020 con ventas mayores a 1.428 millones de dólares y más de 1,5 millones de toneladas de grano canjeadas, y apuesta a incrementar estos resultados en 2021.\n\nEn ese sentido, lanzó su nueva campaña de comunicación orientada a reforzar la idea de circular

En este conjunto de datos tengo las etiquetas de la clasificación realizada con el modelo BETO:

https://huggingface.co/piuba-bigdata/beto-contextualized-hate-speech

Puede usarse para comparar pero en principio nos interesa recuperar los comentarios completos para aplicarle el clasificador seleccionado

In [21]:
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

In [ ]:
tweets_arg = []

for noticia in tqdm(dataset['train']):

    date_noticia = noticia["created_at"]
    # Convertimos la fecha de la noticia a un objeto de python
    datenoticia = datetime.strptime(date_noticia, "%Y-%m-%dT%H:%M:%S%fZ")

    for comment in noticia["comments"]:
        date_comment = comment["created_at"]
        # Convertimos la fecha del comentario a un objeto de python
        datecomment = datetime.strptime(date_comment, "%Y-%m-%dT%H:%M:%S%fZ")
        # anexa comentarios de diarios argentinos
        tweets_arg.append({
            "tweet_id_noticia": noticia["tweet_id"],
            #"user_noticia": noticia["user"],
           # "date_noticia": datenoticia,
           # "title_noticia": noticia["title"],
            #"url_noticia": noticia["url"],
            "tweet": comment["text"],
            "date_tweet": datecomment,
            "APPEARANCE":comment["prediction"]["APPEARANCE"],
            "CLASS":comment["prediction"]["CLASS"],
            "CRIMINAL":comment["prediction"]["CRIMINAL"],
            "DISABLED":comment["prediction"]["DISABLED"],
            "LGBTI":comment["prediction"]["LGBTI"],
            "POLITICS":comment["prediction"]["POLITICS"],
            "RACISM":comment["prediction"]["RACISM"],
            "WOMEN":comment["prediction"]["WOMEN"],
            "CALLS":comment["prediction"]["CALLS"]
            })

  0%|          | 0/537201 [00:00<?, ?it/s]

In [22]:
tweets_arg[123]

{'tweet_id_noticia': '1376941040205238272',
 'tweet': '@elmundoes Nazis. Son nazis.',
 'date_tweet': datetime.datetime(2021, 3, 30, 17, 14, 4, 100000),
 'APPEARANCE': 0,
 'CLASS': 0,
 'CRIMINAL': 0,
 'DISABLED': 0,
 'LGBTI': 0,
 'POLITICS': 1,
 'RACISM': 0,
 'WOMEN': 0,
 'CALLS': 0}

Creo un archivo csv con estos datos para tenerlos por las dudas

In [ ]:
# prompt: save to csv file tweets_arg
#
#import csv

#keys = tweets_arg[0].keys()

#with open('tweets_arg.csv', 'w', newline='') as output_file:
#    dict_writer = csv.DictWriter(output_file, keys)
#    dict_writer.writeheader()
#    dict_writer.writerows(tweets_arg)


In [23]:
# prompt: creare a dataframe of a sample od tweets_arg

import random
# Sample 1000 tweets
sample_tweets = random.sample(tweets_arg, 1000)

# Create dataframe
df_sample = pd.DataFrame(sample_tweets)


In [25]:
df_sample

,tweet_id_noticia,tweet,date_tweet,APPEARANCE,CLASS,CRIMINAL,DISABLED,LGBTI,POLITICS,RACISM,WOMEN,CALLS
0,1284216371447050241,"@LANACION No es conflicto, no delincuentes que...",2020-07-19 12:49:04.500,0,0,0,0,0,0,0,0,0
1,1298740132779094016,@clarincom Bien hecho! https://t.co/NRJ9SfEPuM,2020-08-27 01:04:01.200,0,0,0,0,0,0,0,0,0
2,1255062366741553152,@infobae Existieron siempre. Qué se hacen los ...,2020-04-28 12:57:01.700,0,0,0,0,0,0,0,0,0
3,1285717417575022596,@clarincom Esta todo dado vuelta en Argentina....,2020-07-22 12:12:00.700,0,0,0,0,0,0,0,0,0
4,1245373941889355776,@elpaisuy 60 disparos y los policías no llegar...,2020-04-01 17:00:03.900,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1280594888271302657,@infobae El exito del socialismo,2020-07-08 00:34:03.100,0,0,0,0,0,0,0,0,0
996,1316778960156585984,@infobae Porque no me sorprende,2020-10-15 16:36:00.000,0,0,0,0,0,0,0,0,0
997,1344105867696893952,"@clarincom Asquerosas, por suerte esas enferma...",2020-12-30 23:20:03.000,0,0,0,0,0,0,0,1,0
998,1328672348514476033,@infobae Es asi los pondran en las listas saba...,2020-11-17 12:14:03.400,0,0,0,0,0,0,0,0,0


Aplico el modelo con finetuning de BETO MULTIETIQUETA

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

checkpoint = "natdebandi/beto_hate_speech_ar_multiclasif"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokens = tokenizer(df_sample['tweet'].tolist(), padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)



In [ ]:
outputs[1]

In [ ]:
# prompt: use the transformers librerys of higging face to apply a mult label clasification to the varible tweet in df_sample

!pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "Davlan/bert-base-multilingual-cased-hate-speech-spanish"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Prepare the data for the model
def preprocess_function(examples):
    return tokenizer(examples["tweet"], truncation=True, padding=True)

encoded_dataset = df_sample.map(preprocess_function, batched=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Perform inference
predictions = []
with torch.no_grad():
    for i in tqdm(range(len(df_sample))):
        input_ids = torch.tensor(encoded_dataset['input_ids'][i]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(encoded_dataset['attention_mask'][i]).unsqueeze(0).to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.sigmoid(logits).squeeze().tolist()
        predictions.append(predicted_labels)

# Add predictions to the dataframe
df_sample['predictions'] = predictions
